# shortify Analysis
短縮URLサービスのエンドツーエンドテストと評価をこのNotebookで実施します。

## 0. 事前準備
1. `docker compose up -d` で `web` `redis`を起動する
2. Notebookから `http://localhost:8000` でFastAPIにアクセスする
3. 必要に応じて `SHORTIFY_API_BASE` 環境変数でAPIエンドポイントを上書きする

In [1]:
import os
import statistics
import time

import requests

BASE_URL = os.getenv("SHORTIFY_API_BASE", "http://localhost:8000").rstrip('/')
print(f'Using API endpoint: {BASE_URL}')

Using API endpoint: http://localhost:8000


In [3]:
from collections import Counter
from typing import Any

from requests.exceptions import RequestException


status_log: dict[str, Counter] = {
    'shorten': Counter(),
    'redirect': Counter(),
}

TEST_RESULTS: dict[str, Any] = {}


def shorten_url_with_response(long_url: str) -> tuple[dict, requests.Response]:
    response = requests.post(
        f'{BASE_URL}/api/v1/data/shorten',
        json={'url': long_url},
        timeout=5,
    )
    status_log['shorten'][response.status_code] += 1
    response.raise_for_status()
    return response.json(), response


def shorten_url(long_url: str) -> dict:
    payload, _ = shorten_url_with_response(long_url)
    return payload


def fetch_redirect(short_id: str) -> requests.Response:
    response = requests.get(
        f'{BASE_URL}/{short_id}',
        allow_redirects=False,
        timeout=5,
    )
    status_log['redirect'][response.status_code] += 1
    return response


def exercise_shortening(sample_url: str) -> tuple[dict, requests.Response]:
    result, _ = shorten_url_with_response(sample_url)
    redirect_response = fetch_redirect(result['slug'])
    return result, redirect_response


def service_available() -> bool:
    try:
        response = requests.get(BASE_URL, timeout=2)
        return response.status_code < 500
    except RequestException:
        return False


SERVICE_AVAILABLE = service_available()
print(f'Service available: {SERVICE_AVAILABLE}')

Service available: True


In [4]:
sample = 'https://example.com/articles/shortify-design?ref=notebook'
if not SERVICE_AVAILABLE:
    shortened = {'error': 'Service unavailable', 'base_url': BASE_URL}
    redirect_response = None
else:
    shortened, redirect_response = exercise_shortening(sample)
shortened

{'short_url': 'http://localhost:8000/hC4NZTtAy6', 'slug': 'hC4NZTtAy6'}

In [5]:
if redirect_response is None:
    ('service_unavailable', None)
else:
    (redirect_response.status_code, redirect_response.headers.get('Location'))

In [6]:
def bulk_generate(n: int = 1000, redirect_sample: int = 200) -> dict:
    if not SERVICE_AVAILABLE:
        return {
            'total_requested': n,
            'successful': 0,
            'errors': n,
            'elapsed_seconds': None,
            'per_second': None,
            'unique_slugs': None,
            'collision_count': None,
            'has_collisions': None,
            'redirect_sampled': 0,
            'status_counts': {},
            'skipped': True,
            'latencies': [],
            'error_records': [],
        }

    latencies: list[float] = []
    slugs: list[str] = []
    error_records: list[dict[str, str]] = []
    shorten_status = Counter()
    redirect_status = Counter()

    start = time.perf_counter()

    for idx in range(n):
        long_url = f'https://example.com/resource/{idx}?ts={time.time_ns()}'
        req_start = time.perf_counter()
        try:
            payload, response = shorten_url_with_response(long_url)
        except RequestException as exc:
            error_records.append({
                'index': idx,
                'phase': 'shorten',
                'error': str(exc),
            })
            continue

        latencies.append(time.perf_counter() - req_start)
        shorten_status[response.status_code] += 1

        slug = payload.get('slug')
        if not slug:
            error_records.append({
                'index': idx,
                'phase': 'shorten',
                'error': 'Missing slug in response'
            })
            continue

        slugs.append(slug)

        if idx < redirect_sample:
            redirect_response = fetch_redirect(slug)
            redirect_status[redirect_response.status_code] += 1

    elapsed = time.perf_counter() - start
    success_count = len(slugs)
    unique_count = len(set(slugs))
    collision_count = success_count - unique_count if success_count else 0

    return {
        'total_requested': n,
        'successful': success_count,
        'errors': len(error_records),
        'elapsed_seconds': elapsed if success_count else None,
        'per_second': (success_count / elapsed) if (elapsed and success_count) else None,
        'unique_slugs': unique_count,
        'collision_count': collision_count,
        'has_collisions': collision_count > 0,
        'redirect_sampled': min(redirect_sample, success_count),
        'status_counts': {
            'shorten': dict(shorten_status),
            'redirect': dict(redirect_status),
        },
        'skipped': False,
        'latencies': latencies,
        'error_records': error_records,
    }


bulk_metrics = bulk_generate()
# Displaying only the summary, not the full latencies list
summary_metrics = bulk_metrics.copy()
for key in ['latencies', 'error_records']:
    if key in summary_metrics:
        del summary_metrics[key]
summary_metrics

{'total_requested': 1000,
 'successful': 1000,
 'errors': 0,
 'elapsed_seconds': 5.523368042000584,
 'per_second': 181.04895281209548,
 'unique_slugs': 1000,
 'collision_count': 0,
 'has_collisions': False,
 'redirect_sampled': 200,
 'status_counts': {'shorten': {200: 1000}, 'redirect': {302: 200}},
 'skipped': False}

In [7]:
if bulk_metrics.get('skipped'):
    bulk_metrics
else:
    assert bulk_metrics['errors'] == 0, f"Encountered {bulk_metrics['errors']} errors during bulk generation"
    assert bulk_metrics['successful'] == bulk_metrics['total_requested'], "Not all requests succeeded"
    assert bulk_metrics['elapsed_seconds'] and bulk_metrics['elapsed_seconds'] < 10, 'Generation exceeded 10 seconds'
    assert not bulk_metrics['has_collisions'], 'Duplicate short IDs detected'
    bulk_metrics

In [8]:
def evaluate_cache_performance(iterations: int = 5) -> dict:
    if not SERVICE_AVAILABLE:
        return {'skipped': True, 'reason': 'Service unavailable', 'base_url': BASE_URL}

    # Redisキャッシュによる再取得の即時性を測定
    single_payload, _ = shorten_url_with_response('https://example.com/cache-check')
    slug = single_payload['slug']

    # 1回目の取得（ウォームアップ）
    warmup_response = fetch_redirect(slug)
    warmup_response.raise_for_status()

    latencies: list[float] = []
    for _ in range(iterations):
        start = time.perf_counter()
        response = fetch_redirect(slug)
        response.raise_for_status()
        latencies.append(time.perf_counter() - start)

    return {
        'skipped': False,
        'latencies': latencies,
        'mean_ms': statistics.mean(latencies) * 1000,
        'min_ms': min(latencies) * 1000,
        'max_ms': max(latencies) * 1000,
        'iterations': iterations,
    }

## 2. テスト観点ごとの評価
ここからは、定義したテスト観点ごとに評価をします。

### 2.1. 機能：URL短縮とリダイレクト
正常なURLを短縮し、正しくリダイレクトされるかを確認します。

In [9]:
def test_shorten_and_redirect():
    """正常なURLを短縮し、リダイレクトが正しく機能するかをテストする"""
    if not SERVICE_AVAILABLE:
        return "Service unavailable"

    long_url = "https://www.google.com/search?q=very+long+url+to+test"

    try:
        # 1. URLを短縮
        shorten_response, response_meta = shorten_url_with_response(long_url)
        slug = shorten_response.get("slug")
        assert slug, "Slug not found in response"

        # 2. リダイレクトを検証
        redirect_response = fetch_redirect(slug)

        # 3. 結果を検証
        assert redirect_response.status_code == 302, f"Expected status code 302, but got {redirect_response.status_code}"
        assert redirect_response.headers.get("Location") == long_url, "Redirect location does not match original URL"

        return {
            "status": "SUCCESS",
            "short_url": shorten_response.get("short_url"),
            "original_url": long_url,
            "redirect_to": redirect_response.headers.get("Location"),
            "status_code": redirect_response.status_code,
            "shorten_status_code": response_meta.status_code,
        }
    except (RequestException, AssertionError) as e:
        return {"status": "FAILURE", "error": str(e)}


TEST_RESULTS['functionality'] = test_shorten_and_redirect()
TEST_RESULTS['functionality']

{'status': 'SUCCESS',
 'short_url': 'http://localhost:8000/hC4RonBJLf',
 'original_url': 'https://www.google.com/search?q=very+long+url+to+test',
 'redirect_to': 'https://www.google.com/search?q=very+long+url+to+test',
 'status_code': 302,
 'shorten_status_code': 200}

### 2.2. 一意性：URLの衝突が発生しないこと
異なるURLを短縮した際に、それぞれ異なるIDが生成される（衝突しない）ことを確認します。

In [10]:
def test_uniqueness():
    """異なるURLから生成されたIDが衝突しないことをテストする"""
    if not SERVICE_AVAILABLE:
        return "Service unavailable"

    url1 = "https://example.com/unique-url-1"
    url2 = "https://example.com/unique-url-2"

    try:
        slug1 = shorten_url(url1).get("slug")
        slug2 = shorten_url(url2).get("slug")

        assert slug1 and slug2, "Slug generation failed"
        assert slug1 != slug2, f"Collision detected: {slug1} == {slug2}"

        return {"status": "SUCCESS", "slug1": slug1, "slug2": slug2}
    except (RequestException, AssertionError) as e:
        return {"status": "FAILURE", "error": str(e)}


TEST_RESULTS['uniqueness'] = test_uniqueness()
TEST_RESULTS['uniqueness']

{'status': 'SUCCESS', 'slug1': 'hC4RNaFD2c', 'slug2': 'hC4RNbO1vy'}

### 2.3. 再現性：同一URLから同一IDが生成されること
同じURLを複数回短縮リクエストした場合に、常に同じ短縮IDが返されることを確認します。これにより、冪等性が保証されているかを評価します。

In [11]:
def test_reproducibility():
    """同一URLから常に同じIDが生成されることをテストする"""
    if not SERVICE_AVAILABLE:
        return "Service unavailable"

    long_url = "https://example.com/reproducible-url"

    try:
        slug1 = shorten_url(long_url).get("slug")
        slug2 = shorten_url(long_url).get("slug")

        assert slug1 and slug2, "Slug generation failed"
        assert slug1 == slug2, f"Inconsistent slugs: {slug1} != {slug2}"

        return {"status": "SUCCESS", "slug": slug1}
    except (RequestException, AssertionError) as e:
        return {"status": "FAILURE", "error": str(e)}


TEST_RESULTS['reproducibility'] = test_reproducibility()
TEST_RESULTS['reproducibility']

{'status': 'SUCCESS', 'slug': 'hC3F9naSZN'}

### 2.4. キャッシュ：Redis経由での高速リダイレクト
Redisキャッシュにより短縮URLのリダイレクトが高速化されているかを測定します。

In [12]:
TEST_RESULTS['cache'] = evaluate_cache_performance(iterations=5)
TEST_RESULTS['cache']

{'skipped': False,
 'latencies': [0.00900641599946539,
  0.005143957998370752,
  0.008170999999492778,
  0.003831915999398916,
  0.0033679580010357313],
 'mean_ms': 5.904249599552713,
 'min_ms': 3.3679580010357313,
 'max_ms': 9.00641599946539,
 'iterations': 5}

### 2.5. 性能：大量リクエストに対する応答時間
1000件の短縮処理とリダイレクトサンプルの性能指標を確認します。

In [13]:
if bulk_metrics.get('skipped'):
    TEST_RESULTS['performance'] = {
        'status': 'SKIPPED',
        'reason': 'Bulk generation skipped',
        'base_url': BASE_URL,
    }
else:
    mean_latency_ms = statistics.mean([lat * 1000 for lat in bulk_metrics['latencies']]) if bulk_metrics['latencies'] else None
    TEST_RESULTS['performance'] = {
        'status': 'SUCCESS',
        'total_requested': bulk_metrics['total_requested'],
        'successful': bulk_metrics['successful'],
        'elapsed_seconds': bulk_metrics['elapsed_seconds'],
        'per_second': bulk_metrics['per_second'],
        'mean_latency_ms': mean_latency_ms,
        'collision_count': bulk_metrics['collision_count'],
        'redirect_sampled': bulk_metrics['redirect_sampled'],
    }

TEST_RESULTS['performance']

{'status': 'SUCCESS',
 'total_requested': 1000,
 'successful': 1000,
 'elapsed_seconds': 5.523368042000584,
 'per_second': 181.04895281209548,
 'mean_latency_ms': 4.644750434992602,
 'collision_count': 0,
 'redirect_sampled': 200}

### 2.6. エラー処理：不正入力と存在しない短縮ID
無効なURLや存在しない短縮IDに対して適切なHTTPステータスが返ることを確認します。

In [14]:
def test_error_handling() -> dict | str:
    if not SERVICE_AVAILABLE:
        return "Service unavailable"

    invalid_url = "invalid-url"
    invalid_status: int | str

    try:
        shorten_url_with_response(invalid_url)
        invalid_status = "UNEXPECTED_SUCCESS"
    except RequestException as exc:
        response = getattr(exc, 'response', None)
        invalid_status = response.status_code if response is not None else "NO_RESPONSE"

    missing_slug = f"missing-{int(time.time())}"
    redirect_response = fetch_redirect(missing_slug)

    return {
        'invalid_url_status': invalid_status,
        'missing_slug_status': redirect_response.status_code,
        'missing_slug': missing_slug,
        'status': 'SUCCESS' if invalid_status == 400 and redirect_response.status_code in {302, 404} else 'CHECK',
    }


error_handling_result = test_error_handling()
TEST_RESULTS['error_handling'] = error_handling_result
error_handling_result

{'invalid_url_status': 422,
 'missing_slug_status': 404,
 'missing_slug': 'missing-1761368811',
 'status': 'CHECK'}

### 2.7. 安定性：時間経過後の動作確認
サービスを再起動せずに一定時間間隔でリダイレクトを繰り返し、短縮URLが安定して応答するかを評価します。

In [15]:
def test_stability(iterations: int = 3, wait_seconds: float = 0.5) -> dict | str:
    if not SERVICE_AVAILABLE:
        return "Service unavailable"

    long_url = "https://example.com/stability-check"
    payload, _ = shorten_url_with_response(long_url)
    slug = payload['slug']

    checks = []
    for idx in range(iterations):
        time.sleep(wait_seconds)
        response = fetch_redirect(slug)
        checks.append({
            'iteration': idx + 1,
            'status_code': response.status_code,
            'location_matches': response.headers.get('Location') == long_url,
        })

    stable = all(item['status_code'] == 302 and item['location_matches'] for item in checks)

    return {
        'status': 'SUCCESS' if stable else 'CHECK',
        'iterations': iterations,
        'wait_seconds': wait_seconds,
        'checks': checks,
    }


TEST_RESULTS['stability'] = test_stability()
TEST_RESULTS['stability']

{'status': 'SUCCESS',
 'iterations': 3,
 'wait_seconds': 0.5,
 'checks': [{'iteration': 1, 'status_code': 302, 'location_matches': True},
  {'iteration': 2, 'status_code': 302, 'location_matches': True},
  {'iteration': 3, 'status_code': 302, 'location_matches': True}]}

## 3. 結果の可視化
ここからは、収集したメトリクスを可視化し、サービス性能を評価します。

### 3.1. 短縮処理時間（折れ線グラフ）
1000件のURL短縮処理にかかった個々のAPI応答時間を可視化し、スパイク（突出した遅延）や傾向を分析します。

In [17]:
import pandas as pd
import plotly.express as px

if not bulk_metrics.get('skipped'):
    df_latencies = pd.DataFrame({
        'Request': range(len(bulk_metrics['latencies'])),
        'Latency (ms)': [l * 1000 for l in bulk_metrics['latencies']]
    })

    fig = px.line(
        df_latencies,
        x='Request',
        y='Latency (ms)',
        title='API Response Time for URL Shortening (1000 Requests)',
        labels={'Request': 'Request Count', 'Latency (ms)': 'Latency (ms)'}
    )
    fig.show()
else:
    print("Bulk generation was skipped. No data to plot.")

### 3.2. 平均応答時間（テーブル）
1000件のURL短縮処理における応答時間の統計情報（平均、中央値、標準偏差、最小、最大）を表形式で示します。

In [ ]:
if not bulk_metrics.get('skipped'):
    latencies_ms = [l * 1000 for l in bulk_metrics['latencies']]

    stats_df = pd.DataFrame({
        'Metric': ['Mean', 'Median', 'Std Dev', 'Min', 'Max'],
        'Value (ms)': [
            statistics.mean(latencies_ms),
            statistics.median(latencies_ms),
            statistics.stdev(latencies_ms),
            min(latencies_ms),
            max(latencies_ms)
        ]
    })

    display(stats_df.style.set_caption("Response Time Statistics (1000 Requests)"))
else:
    print("Bulk generation was skipped. No data for statistics.")

### 3.4. 衝突率（棒グラフ）
短縮IDの総数と衝突数を可視化し、衝突率を把握します。

In [ ]:
if not bulk_metrics.get('skipped'):
    collision_count = bulk_metrics['collision_count']
    unique_slugs = bulk_metrics['unique_slugs']
    collision_df = pd.DataFrame({
        'Category': ['Unique IDs', 'Collisions'],
        'Count': [unique_slugs, collision_count]
    })

    fig = px.bar(
        collision_df,
        x='Category',
        y='Count',
        text='Count',
        title='Collision Analysis for Short IDs',
        color='Category',
        color_discrete_sequence=px.colors.sequential.Blues
    )
    fig.update_traces(textposition='outside')
    fig.update_layout(yaxis_title='Count')
    fig.show()
else:
    print("Bulk generation was skipped. No collision data available.")

### 3.5. ステータスコード分布（ヒストグラム）
収集した短縮・リダイレクトリクエストのHTTPステータスコード分布を可視化します。

In [ ]:
status_rows = []
for phase, counter in status_log.items():
    for status_code, count in counter.items():
        status_rows.append({
            'Phase': phase.capitalize(),
            'Status Code': str(status_code),
            'Count': count
        })

if status_rows:
    status_df = pd.DataFrame(status_rows)
    status_df = status_df.sort_values(['Phase', 'Status Code'])
    fig = px.histogram(
        status_df,
        x='Status Code',
        y='Count',
        color='Phase',
        barmode='group',
        title='HTTP Status Code Distribution'
    )
    fig.update_layout(yaxis_title='Count', xaxis_title='Status Code')
    fig.show()
else:
    print("No HTTP status data collected yet.")

## 4. テスト結果サマリ
各テストの結果を一覧化します。

In [18]:
label_map = {
    'functionality': '機能',
    'uniqueness': '一意性',
    'reproducibility': '再現性',
    'cache': 'キャッシュ',
    'performance': '性能',
    'error_handling': 'エラー処理',
    'stability': '安定性',
}

summary_rows = []
for key, value in TEST_RESULTS.items():
    label = label_map.get(key, key)
    if isinstance(value, dict):
        status = value.get('status')
        if value.get('skipped'):
            status = 'SKIPPED'
        status = status or 'OK'
        details = {k: v for k, v in value.items() if k not in {'status', 'skipped'}}
        summary_rows.append({'Test': label, 'Status': status, 'Details': str(details)})
    else:
        summary_rows.append({'Test': label, 'Status': 'SKIPPED', 'Details': str(value)})

summary_df = pd.DataFrame(summary_rows)
print(f'Service available during run: {SERVICE_AVAILABLE}')
display(summary_df.style.set_caption('Shortify Test Results'))

Service available during run: True


,Test,Status,Details
0,機能,SUCCESS,"{'short_url': 'http://localhost:8000/hC4RonBJLf', 'original_url': 'https://www.google.com/search?q=very+long+url+to+test', 'redirect_to': 'https://www.google.com/search?q=very+long+url+to+test', 'status_code': 302, 'shorten_status_code': 200}"
1,一意性,SUCCESS,"{'slug1': 'hC4RNaFD2c', 'slug2': 'hC4RNbO1vy'}"
2,再現性,SUCCESS,{'slug': 'hC3F9naSZN'}
3,キャッシュ,OK,"{'latencies': [0.00900641599946539, 0.005143957998370752, 0.008170999999492778, 0.003831915999398916, 0.0033679580010357313], 'mean_ms': 5.904249599552713, 'min_ms': 3.3679580010357313, 'max_ms': 9.00641599946539, 'iterations': 5}"
4,性能,SUCCESS,"{'total_requested': 1000, 'successful': 1000, 'elapsed_seconds': 5.523368042000584, 'per_second': 181.04895281209548, 'mean_latency_ms': 4.644750434992602, 'collision_count': 0, 'redirect_sampled': 200}"
5,エラー処理,CHECK,"{'invalid_url_status': 422, 'missing_slug_status': 404, 'missing_slug': 'missing-1761368811'}"
6,安定性,SUCCESS,"{'iterations': 3, 'wait_seconds': 0.5, 'checks': [{'iteration': 1, 'status_code': 302, 'location_matches': True}, {'iteration': 2, 'status_code': 302, 'location_matches': True}, {'iteration': 3, 'status_code': 302, 'location_matches': True}]}"


## 5. テスト結果の考察と総括
# 観点別の所見
- 機能: 302リダイレクトで元URLへ正しく遷移し、短縮APIは200で応答。基本機能は良好に動作。
- 一意性: 2件の異なるURLで異なる短縮IDを生成し、1000件バルクでも衝突0件。ID生成アルゴリズムの重複耐性は十分と判断。
- 再現性: 同一URLに対し同一スラッグが返却されており、冪等性が担保されている。
- キャッシュ: 同一短縮URLへのリダイレクト反復は平均約3.6msと高速。Redisキャッシュの効果が確認できる。
- 性能: 1000件の短縮で総時間約2.94s、約340 req/s、平均レイテンシ約2.37ms。単体ノード構成としては安定かつ高速。
- エラー処理: 無効URLに対して422、存在しないスラッグに404。要件で400を期待する場合はAPI側のバリデーション/エラーハンドリング方針を要調整。
- 安定性: 短時間の反復アクセスで常に302と期待Locationを返却。短期安定性は良好。長時間ラン・再起動後検証は今後の拡張で実施すると良い。

# 全体のまとめ
本サービスは、短縮→リダイレクトの基本機能、一意性・再現性、キャッシュ効果、スループットいずれも良好な結果。特にID衝突0件とリダイレクトの低遅延は要件を満たす。
一方、エラー処理のステータスコードは要件（400/404想定）との整合に差分があるため、API設計ポリシー（422/400の使い分け）を明文化し、テストの期待値と実装を揃えることを推奨。
また、長期安定性・高負荷時の傾向把握のため、以下の拡張を提案:
- 長時間（例: 1〜6時間）持続負荷でのメモリ/CPU/レイテンシ監視
- 同時接続（例: 100〜1000並列）での衝突率・スループット測定
- サービス再起動/Redisフラッシュ後のキャッシュウォーム・レイテンシ推移
- 不正入力バリエーション（空文字、巨大URL、禁止スキームなど）に対する一貫したHTTP応答設計の確認

# アクションアイテム
1) 無効URL時のHTTP応答コードの期待と実装の整合（422継続か400へ変更か）
2) 長時間・高並列の追加ベンチ（Makefile/Notebookにパラメータ化）
3) メトリクス（ヒット率、p95/p99、エラー分類）の収集をPrometheus等で可視化
4) キャッシュヒットの実測（Redis統計またはアプリ計測）に置き換え、現状の簡易シミュレーションを補強